[dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

[Bonne segmentation des clients](https://www.kaggle.com/code/farrasalyafi/eda-customer-segmentatioin)


In [1]:
url = 'https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/download'
zip_file_name = 'brazilian-e-commerce-dataset.zip'
extract_name = 'olist_customers_dataset.csv'
dl_path = 'downloads'
zip_file_path = f'./{dl_path}/{zip_file_name}'
extracts_path = 'extracts'
extract_to_path = f'./{extracts_path}/{extract_name}'

In [2]:
if os.path.isfile(zip_file_path):
    print('File already downloaded')
else:
    # Create the downloads folder
    os.makedirs(dl_path)

    # Download file from url
    import urllib.request
    urllib.request.urlretrieve(url, zip_file_path)
    print('File downloaded')

if os.path.isfile(extract_to_path):
    print('File already extracted')
else:
    # Unzip file
    import zipfile
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extracts_path)
    print('Files extracted')

NameError: name 'os' is not defined

Using this schema to navigate between the different datasets :
![dataset_relation_schema](https://i.imgur.com/HRhd2Y0.png)

As we have to segment clients profiles, we will mostly use the client dataset (olist_order_customer_dataset) related to others.

# Exploration

In order to avoid to explore every dataset, we will do only for data_customer then add a dataset to the exploration part every time we need one.

## Retrieving the data

In [ ]:
import pandas as pd

# Reading all the files
raw_path = './extracts/'
data_customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
data_geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
data_orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
data_order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
data_order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
data_order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
data_products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
data_sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')
data_category = pd.read_csv(raw_path + 'product_category_name_translation.csv')

In [ ]:
#Menggabungkan semua data-data 
all_data = data_orders.merge(data_order_items, on='order_id', how='left')
all_data = all_data.merge(data_order_payments, on='order_id', how='inner')
all_data = all_data.merge(data_order_reviews, on='order_id', how='inner')
all_data = all_data.merge(data_products, on='product_id', how='inner')
all_data = all_data.merge(data_customer, on='customer_id', how='inner')
all_data = all_data.merge(data_sellers, on='seller_id', how='inner')
all_data = all_data.merge(data_category,on='product_category_name',how='inner')
all_data.dtypes

## 1st : data_customer

### Shape

- `customer_id` : key to the orders dataset. Each order has a unique customer_id.
- `customer_unique_id` : unique identifier of a customer.
- `customer_zip_code_prefix` : first five digits of customer zip code.
- `customer_city` : customer city name.
- `customer_state` : customer state.

In [ ]:
print(data_customer.dtypes)
data_customer.head()

In [ ]:
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="whitegrid")

number_of_orders = Counter(data_customer['customer_unique_id'].value_counts().values)
pd_number_of_orders = pd.DataFrame(number_of_orders.items(), columns=['number_of_order_per_customer', 'occurences'])
# keys = number_of_orders.keys()
# values = number_of_orders.values()

ax = sns.barplot(x='number_of_order_per_customer', y = 'occurences', data = pd_number_of_orders)
ax.set(xlabel='Number of orders', ylabel='Occurences')
ax.set_title('Number of orders per customer frequence')
ax.set_ylim(0, 100000)
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
data_customer[data_customer.isnull().sum(axis=1) != 0]

No missing values.

### Oultiers

Ids seemed to be in hex with a length of 32, we check that they all fulfill this condition.

In [ ]:
import string

if data_customer.loc[data_customer['customer_unique_id'].str.len() != 32,:].shape[0] == 0:
    print('All the customer_unique_id are 32 characters long')
else:
    print('There are some customer_unique_id which are not 32 characters long')

try:
    data_customer['customer_unique_id'].apply(int, base=16)
    print('All the customer_unique_id are in base 16')
except:
    print('There are some customer_unique_id which are not in base 16')
    
if data_customer.loc[data_customer['customer_id'].str.len() != 32,:].shape[0] == 0:
    print('All the customer_id are 32 characters long')
else:
    print('There are some customer_id which are not 32 characters long')

try:
    data_customer['customer_id'].apply(int, base=16)
    print('All the customer_id are in base 16')
except:
    print('There are some customer_id which are not in base 16')

The other columns on the dataset are not interesting.

## 2nd : data_orders

### Shape

- `order_id` : unique identifier of the order.
- `customer_id` : key to the customer dataset. Each order has a unique customer_id.
- `order_status` : Reference to the order status (delivered, shipped, etc).
- `order_purchase_timestamp` : Shows the purchase timestamp.
- `order_approved_at` : Shows the payment approval timestamp.
- `order_delivered_carrier_date` : Shows the order posting timestamp. When it was handled to the logistic partner.
- `order_delivered_customer_date` : Shows the actual order delivery date to the customer.
- `order_estimated_delivery_date` : Shows the estimated delivery date that was informed to customer at the purchase moment.

In [ ]:
print(data_orders.dtypes)
data_orders.head()

There are a lot of dates in this dataframe, we will check their distribution by grouping them per month.

In [ ]:
data_orders_date_columns = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
data_order_visualization = data_orders[data_orders_date_columns]


fig, axs = plt.subplots(len(data_orders_date_columns),3,gridspec_kw={'width_ratios': [2, 4, 2]}, figsize=(12,15))

for i, column in enumerate(data_orders_date_columns):
    
    sorted_dates = pd.to_datetime(data_order_visualization[column]).dt.strftime(' %y-%m').dropna()
    sorted_dates = sorted_dates.sort_values(ascending=True)
    hundred_first_dates = sorted_dates.head(round(len(sorted_dates)*0.1))
    hundred_last_dates = sorted_dates.tail(round(len(sorted_dates)*0.1))

    axs[i,0].hist(hundred_first_dates, bins=20)
    axs[i,0].set_title('First 10% of ' + column)
    axs[i,0].set_xlabel(column)
    axs[i,0].tick_params(axis='x', rotation=90)
    
    axs[i,1].hist(sorted_dates, bins=40)
    axs[i,1].set_title('All dates of ' + column)
    axs[i,1].set_xlabel(column)
    axs[i,1].set_ylabel('Frequency')
    axs[i,1].tick_params(axis='x', rotation=90)

    axs[i,2].hist(hundred_last_dates, bins=20)
    axs[i,2].set_title('Last 10% of ' + column)
    axs[i,2].set_xlabel(column)
    axs[i,2].set_ylabel('Frequency')
    axs[i,2].tick_params(axis='x', rotation=90)

plt.tight_layout()
print(fig)

### Oultiers

I think it could be reasonneable to truncate the date from 01 Mar 2017 to 30 Sep 2018.

In [ ]:
min_datetime = pd.to_datetime('2017-03-01')
max_datetime = pd.to_datetime('2018-09-30')

# Convert the date to datetime format
data_orders[data_orders_date_columns] = data_orders[data_orders_date_columns].apply(pd.to_datetime)

# If one of the colums is over the max date or under the min date, then we delete the order
conditions = (
    (data_orders['order_purchase_timestamp'] > max_datetime)
    | (pd.to_datetime(data_orders['order_purchase_timestamp']) <  min_datetime)
    | (data_orders['order_approved_at'] > max_datetime)
    | (pd.to_datetime(data_orders['order_approved_at']) <  min_datetime)
    | (data_orders['order_delivered_carrier_date'] > max_datetime)
    | (pd.to_datetime(data_orders['order_delivered_carrier_date']) <  min_datetime)
    | (data_orders['order_delivered_customer_date'] > max_datetime)
    | (pd.to_datetime(data_orders['order_delivered_customer_date']) <  min_datetime)
    | (data_orders['order_estimated_delivery_date'] > max_datetime)
    | (pd.to_datetime(data_orders['order_estimated_delivery_date']) <  min_datetime)
)

orders_to_delete = data_orders[conditions]
print(f"{len(orders_to_delete)} orders will be deleted")
data_orders.drop(orders_to_delete.index, inplace=True)

In [ ]:
data_order_visualization = data_orders[data_orders_date_columns]


fig, axs = plt.subplots(len(data_orders_date_columns),3,gridspec_kw={'width_ratios': [2, 4, 2]}, figsize=(12,15))

for i, column in enumerate(data_orders_date_columns):
    
    sorted_dates = pd.to_datetime(data_order_visualization[column]).dt.strftime(' %y-%m').dropna()
    sorted_dates = sorted_dates.sort_values(ascending=True)
    hundred_first_dates = sorted_dates.head(round(len(sorted_dates)*0.1))
    hundred_last_dates = sorted_dates.tail(round(len(sorted_dates)*0.1))

    axs[i,0].hist(hundred_first_dates, bins=20)
    axs[i,0].set_title('First 10% of ' + column)
    axs[i,0].set_xlabel(column)
    axs[i,0].tick_params(axis='x', rotation=90)

    axs[i,1].hist(sorted_dates, bins=40)
    axs[i,1].set_title('All dates of ' + column)
    axs[i,1].set_xlabel(column)
    axs[i,1].set_ylabel('Frequency')
    axs[i,1].tick_params(axis='x', rotation=90)

    axs[i,2].hist(hundred_last_dates, bins=20)
    axs[i,2].set_title('Last 10% of ' + column)
    axs[i,2].set_xlabel(column)
    axs[i,2].set_ylabel('Frequency')
    axs[i,2].tick_params(axis='x', rotation=90)

plt.tight_layout()
print(fig)

In [ ]:
data_orders

In [ ]:
if data_orders.loc[data_orders['order_id'].str.len() != 32,:].shape[0] == 0:
    print('All the order_id are 32 characters long')
else:
    print('There are some order_id which are not 32 characters long')

try:
    data_orders['order_id'].apply(int, base=16)
    print('All the order_id are in base 16')
except:
    print('There are some order_id which are not in base 16')
    
if data_orders.loc[data_orders['customer_id'].str.len() != 32,:].shape[0] == 0:
    print('All the customer_id are 32 characters long')
else:
    print('There are some customer_id which are not 32 characters long')

try:
    data_orders['customer_id'].apply(int, base=16)
    print('All the customer_id are in base 16')
except:
    print('There are some customer_id which are not in base 16')

## 3rd : data_order_payments

### Shape

- `order_id` : unique identifier of the order.
- `payment_sequential` : a customer may pay an order with more than one payment method. If he does so, a sequence will be created to accommodate all payments.
- `payment_type` : method of payment chosen by the customer.
- `payment_installments` : number of installments chosen by the customer.
- `payment_value` : transaction value.

In [ ]:
print(data_order_payments.dtypes)
data_order_payments.head()

In [ ]:
sequential_payments = data_order_payments['payment_sequential'].value_counts()
fig, ax = plt.subplots(figsize=(12,6))
barplot = ax.bar(x = sequential_payments.index, height=sequential_payments.values)
ax.bar_label(barplot)
ax.set_title('Payment sequential')
ax.set_xlabel('Payment sequential')
ax.set_ylabel('Frequency')
ax.set_xlim(0,30)
ax.set_xticks(sequential_payments.index)
plt.show()

In [ ]:
import numpy as np

to_pie_plot_columns = ['payment_sequential', 'payment_type', 'payment_installments']
others_percent_threshold = 3

fig, ax = plt.subplots(1, len(to_pie_plot_columns), figsize=(12,6))
for i, column in enumerate(to_pie_plot_columns):
    values_counts = data_order_payments[column].value_counts()
    total_values = values_counts.sum()
    others_index = values_counts[values_counts/total_values*100 < others_percent_threshold].index
    values_others = values_counts[others_index]
    values_counts = values_counts.drop(others_index)
    others_sum = values_others.sum()

    values_counts = values_counts.reset_index()
    values_counts['index'] = values_counts['index'].astype(str)
    values_counts = values_counts.append({'index':'others', column:others_sum}, ignore_index=True)


    ax[i].pie(x=values_counts[column], labels =values_counts['index'] , autopct='%1.1f%%')
    ax[i].set_title(column)
plt.show()

In [ ]:
fig, axs = plt.subplots(1,3,gridspec_kw={'width_ratios': [2, 4, 2]}, figsize=(12,4))

payments_sorted = data_order_payments['payment_value'].sort_values(ascending=True)
nb_payments = len(payments_sorted)
ten_percent_first = payments_sorted.head(round(0.1 * nb_payments))
ten_percent_last = payments_sorted.tail(round(0.1 * nb_payments))

axs[0].hist(ten_percent_first, bins=20)
axs[0].set_title('10% lowest payments')
axs[0].tick_params(rotation=90)
axs[0].set_xlim(0, max(ten_percent_first))

axs[1].hist(payments_sorted, bins=40)
axs[1].set_title('All payments')
axs[1].tick_params(rotation=90)
axs[1].set_xlim(min(payments_sorted), max(payments_sorted))

axs[2].hist(ten_percent_last, bins=20)
axs[2].set_title('10% highests payments')
axs[2].tick_params(rotation=90)
axs[2].set_xlim(min(ten_percent_last), max(ten_percent_last))

plt.tight_layout()

# Feature extraction

Feuille de route :

1. Faire un premier jeu de variables avec RFM --> Date de la dernière commande | Nombre de commandes | Total dépensé
2. Faire le clustering sur ces variables sur une période fixe (toute l'année) puis du flotant
3. Rajouter les features et refaire un clustering
4. Rajouter plusieurs algorithmes
5. Faire de l'interpretation en pensant au point de vue marketing

Features interessantes :
- Temps médian entre les commandes
- Somme médianne des commandes
- Mois avec le plus de commandes (avec une pondération ?)
- Catégorie préférée (attention, variable catégorielle)
- Score moyen des reviews
- "A laissé une review"

Adding new features to a custom dataframe representing a customer.
Adding :
- **Recency** (How recently did the customer purchase?) : When was the last order
- **Frequency** (How often do they purchase?) : Average time between orders (time signed up / number of orders)
- **Monetary Value** (How much do they spend?) : What is the median of the order value
- **number of orders** : How many orders the customer has made.
- **customer's seniority** : How long the customer has been a customer.
- **total spent** : How much the customer has spent
- **prefered category** : What is the prefered category of the customer
- **most spent month** : Month with the most orders
- **average review score** : Average review score of the customer
  
Every date is transformed into a number of days from now. And we consider (even if it isn't mentionned) that the times are in UTC.
Also, in order to calculate the customer's seniority, we consider the current date as the 01/01/2019 because the dataset gathers data from 2016 to 2018.

In [ ]:
custom_customers_df = data_customer.drop(['customer_zip_code_prefix', 'customer_city', 'customer_state', 'customer_id'], axis=1).drop_duplicates()

## Recency

**Recency** (How recently did the customer purchase?) : When was the last order

In [ ]:
from datetime import datetime

# Keeping the relation from order customer id and customer unique id
custom_orders_df = data_customer[['customer_unique_id', 'customer_id']]
# Getting the orders details
custom_orders_df = custom_orders_df.merge(data_orders, on='customer_id', how='inner')
# Grouping every customer by their customer_unique_id and selecting the most recent order date (and renaming it)
custom_orders_most_recent_df = custom_orders_df.groupby('customer_unique_id').agg({'order_purchase_timestamp': 'max'}).reset_index()
custom_orders_most_recent_df['Recency'] = (max_datetime - pd.to_datetime(custom_orders_most_recent_df['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')).dt.days
custom_orders_most_recent_df = custom_orders_most_recent_df.drop('order_purchase_timestamp', axis=1)

custom_orders_most_recent_df.sort_values(by='Recency')

# Adding recency column to the custom_customers_df
custom_customers_df = custom_customers_df.merge(custom_orders_most_recent_df, on='customer_unique_id', how='inner')


custom_customers_df.sort_values(by='Recency')

## Frequency

In [ ]:
orders_per_customer = custom_orders_df['customer_unique_id'].value_counts()
orders_per_customer = pd.DataFrame(orders_per_customer.items(), columns=['customer_unique_id', 'number_of_orders'])
custom_customers_df = custom_customers_df.merge(orders_per_customer, on='customer_unique_id', how='inner')
custom_customers_df

# Monetary value

In [ ]:
custom_orders_df[['customer_unique_id']]

In [ ]:
custom_customers_df

In [ ]:
total_spent_per_customer = custom_orders_df['customer_unique_id'].value_counts()
total_spent_per_customer = pd.DataFrame(total_spent_per_customer.items(), columns=['customer_unique_id', 'number_of_orders'])
custom_customers_df = custom_customers_df.merge(orders_per_customer, on='customer_unique_id', how='inner')
custom_customers_df

# Custom features

## Frequency (and seniority and number of orders)

**Frequency** (How often do they purchase?) : Average time between orders (customer's seniority / number of orders)

**Number of orders** : How many orders the customer has made.

**Customer's seniority** : How long the customer has been a customer.

Because there is no information about one customer's first activity, we will use the first order date as the customer's first activity.

In [ ]:
# Reuse of the previous function for getting the most recent order date, but for the oldest order
custom_orders_most_ancient_df = custom_orders_df.groupby('customer_unique_id').agg({'order_purchase_timestamp': 'min'}).reset_index().rename(columns={'order_purchase_timestamp': 'Seniority'})
custom_customers_df = custom_customers_df.merge(custom_orders_most_ancient_df, on='customer_unique_id', how='inner')
custom_customers_df

In [ ]:
orders_per_customer = custom_orders_df['customer_unique_id'].value_counts()
orders_per_customer = pd.DataFrame(orders_per_customer.items(), columns=['customer_unique_id', 'number_of_orders'])
custom_customers_df = custom_customers_df.merge(orders_per_customer, on='customer_unique_id', how='inner')
custom_customers_df